In [1]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import *

In [2]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

# Will get traces from two videos:
btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)
#btf_fname_green = r'test2020-10-22_16-15-20_test4-channel-1-pco_camera2\test2020-10-22_16-15-20_test4-channel-1-pco_camera2bigtiff.btf'
#btf_fname_green = os.path.join(bigtiff_folder, btf_fname_green)

In [16]:
vol0 = get_single_volume(btf_fname_red, 0, num_slices=33, alpha=0.15)

In [17]:
alpha = 0.15
dat_foldername = r'..\point_cloud_alignment'
fname = os.path.join(dat_foldername, 'img100.tif')
f = lambda tif : (alpha*tif.asarray()).astype('uint8')
with tifffile.TiffFile(fname) as tif0:
    vol0_old = f(tif0)

## Use single function to track

In [20]:
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *


In [24]:
all_matches, all_conf = track_neurons_full_video(btf_fname_red,
                             start_frame=100,
                             num_frames=10,
                             num_slices=33,
                             alpha=0.15,
                             verbose=2)

Matching frames 100 and 101 (end at 110)


error: OpenCV(3.4.2) C:\Miniconda3\conda-bld\opencv-suite_1534379934306\work\modules\core\src\batch_distance.cpp:238: error: (-215:Assertion failed) type == src2.type() && src1.cols == src2.cols && (type == 5 || type == 0) in function 'cv::batchDistance'


In [ ]:
%debug


> c:\users\charles.fieseler\documents\current_work\dlc_for_wbfm\dlc_for_wbfm\utils\feature_detection\utils_features.py(39)detect_features_and_match()
     37     # Match features.
     38     matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
---> 39     matches = matcher.match(descriptors1, descriptors2, None)
     40 
     41     # Sort matches by score

ipdb> len(keypoints1)
4
ipdb> len(keypoints2)
0


# Aside: timing tests

In [15]:
# Open video
def open_tif(vid_fname):
    with tifffile.TiffFile(vid_fname) as tif:
        1+1

def open_page(vid_fname):
    with tifffile.TiffFile(vid_fname) as tif:
        for i, page in enumerate(tif.pages):
            dat = page.asarray()
            if i>0:
                break

def open_volume(vid_fname, num_slices=33):
    dat = []
    with tifffile.TiffFile(vid_fname) as tif:
        for i, page in enumerate(tif.pages):
            dat.append(page.asarray())
            if i>num_slices:
                break
                
def open_with_imread(vid_fname, num_slices=33):
    dat = tifffile.imread(vid_fname, key=range(num_slices))

In [9]:
%timeit open_tif(btf_fname_red)

269 µs ± 4.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%timeit open_page(btf_fname_red)

2 ms ± 32.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%timeit open_volume(btf_fname_red)

38.8 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit open_with_imread(btf_fname_red)

31.5 ms ± 4.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
